# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-68.3000,-54.8000,276.96,93,100,1.54,AR,2023-04-24
1,1,hithadhoo,73.0833,-0.6000,302.13,75,99,3.95,MV,2023-04-24
2,2,albany,-73.9662,42.6001,285.14,50,88,2.82,US,2023-04-24
3,3,clyde river,-68.5914,70.4692,268.28,93,75,7.72,CA,2023-04-24
4,4,saint-francois,3.9054,46.4154,283.13,83,100,2.06,FR,2023-04-24


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = "Lat",
    y = "Lng",
    hover_cols = "City",
    geo = True,
    tiles = "OSM",
    color="City",
    legend="right",
    alpha = 0.5
)

# Display the map
map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
clear_sky_warm_df = city_data_df[(pd.to_numeric(city_data_df["Cloudiness"])) == 0 & (pd.to_numeric(city_data_df["Max Temp"]) > 270)]

# Drop any rows with null values
clear_sky_warm_df.dropna(how = "any")

# Display sample data
clear_sky_warm_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,laguna,-121.4238,38.4210,300.73,40,0,2.06,US,2023-04-24
7,7,marsa matruh,27.2453,31.3525,291.26,77,0,3.09,EG,2023-04-24
31,31,viedma,-62.9967,-40.8135,292.67,36,0,4.78,AR,2023-04-24
34,34,tura,90.2201,25.5198,294.24,57,0,1.61,IN,2023-04-24
39,39,makakilo city,-158.0858,21.3469,301.33,68,0,3.60,US,2023-04-24


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clear_sky_warm_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(1000)

,City,Country,Lat,Lng,Humidity,Hotel Name
6,laguna,US,-121.4238,38.4210,40,
7,marsa matruh,EG,27.2453,31.3525,77,
31,viedma,AR,-62.9967,-40.8135,36,
34,tura,IN,90.2201,25.5198,57,
39,makakilo city,US,-158.0858,21.3469,68,
41,saint george,US,-113.5841,37.1041,20,
54,longyearbyen,SJ,15.6401,78.2186,79,
56,taoudenni,ML,-3.9836,22.6783,12,
60,jimo,CN,120.4622,36.3897,46,
67,saint-pierre,RE,55.4781,-21.3393,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
lat = ""
lon = ""
params = {
    "categories": "accommodation.hotel",
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    #Had to flip the lattude and longitude in the variables as the API was not returning any data. The API documenttion is incorrect so had to flip the inputs.
    latitude = row["Lng"]
    longitude =row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"]=f"circle:{longitude},{latitude},{radius}"
    params["bias"]=f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
laguna - nearest hotel: Holiday Inn Express & Suites
marsa matruh - nearest hotel: صقور
viedma - nearest hotel: Níjar
tura - nearest hotel: No hotel found
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
longyearbyen - nearest hotel: The Vault
taoudenni - nearest hotel: No hotel found
jimo - nearest hotel: DoubleTree by Hilton Qingdao
saint-pierre - nearest hotel: Tropic Hotel
providencia - nearest hotel: No hotel found
nome - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
luderitz - nearest hotel: Seaview Hotel Zum Sperrgebiet
mashhad - nearest hotel: هتل تعاون
alice springs - nearest hotel: Aurora Alice Springs
kodiak - nearest hotel: Shelikof Lodge
san quintin - nearest hotel: Jardines Hotel
chik - nearest hotel: No hotel found
comodoro rivadavia - nearest hotel: No hotel found
naraura - nearest hotel: No hotel found
tromso - nearest hotel: Cl

,City,Country,Lat,Lng,Humidity,Hotel Name
6,laguna,US,-121.4238,38.4210,40,Holiday Inn Express & Suites
7,marsa matruh,EG,27.2453,31.3525,77,صقور
31,viedma,AR,-62.9967,-40.8135,36,Níjar
34,tura,IN,90.2201,25.5198,57,No hotel found
39,makakilo city,US,-158.0858,21.3469,68,Embassy Suites by Hilton Oahu Kapolei
41,saint george,US,-113.5841,37.1041,20,"The Advenire, an Autograph Collection Hotel"
54,longyearbyen,SJ,15.6401,78.2186,79,The Vault
56,taoudenni,ML,-3.9836,22.6783,12,No hotel found
60,jimo,CN,120.4622,36.3897,46,DoubleTree by Hilton Qingdao
67,saint-pierre,RE,55.4781,-21.3393,73,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    x = "Lat",
    y = "Lng",
    hover_cols = ["City","Hotel Name","Country"],
    geo = True,
    tiles = "OSM",
    color="City",
    legend="right",
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]   (City,Hotel Name,Country)